In [1]:
#module 불러오기

from supervised import AutoML


import os
import re

import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.manifold import Isomap
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.cluster import KMeans
from sklearn.impute import KNNImputer
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import umap.umap_ as umap
import math as math
import datetime

# keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping

#silhouette score

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import csv

#validation

from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics

## **함수 정리**

In [2]:
#함수 정리

# data.frame에서 column 별 outlier 제거 위한 값 계산
def determine_outlier_thresholds_iqr(dataframe, col_name, th1=0.05, th3=0.95):
    quartile1 = dataframe[col_name].quantile(th1)
    quartile3 = dataframe[col_name].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr
    return lower_limit, upper_limit


# 위의 outlier를 토대로 값 check
def check_outliers_iqr(dataframe, col_name):
    lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)].any(axis=None):
        return True
    else: 
        return False

#  outlier 넘는 값들을 최대값 또는 최소값으로 치환
def replace_with_thresholds_iqr(dataframe, cols, th1=0.05, th3=0.95, replace=False):
    from tabulate import tabulate
    data = []
    for col_name in cols:
        if col_name != 'Outcome':
            outliers_ = check_outliers_iqr(dataframe,col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name, th1, th3)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][col_name].count()
                if replace: 
                    dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                    dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_iqr(dataframe, col_name)
            data.append([outliers_, outliers_status, count, col_name, lower_limit, upper_limit ])
    table = tabulate(data, headers=['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
   
    return dataframe

def cancer_select(cols, cancer_type):
    # phenotype
    phe1 = pd.read_csv("https://gdc-hub.s3.us-east-1.amazonaws.com/download/GDC-PANCAN.basic_phenotype.tsv.gz", sep="\t")
    phe1 = phe1.loc[phe1.program == "TCGA", :].loc[:, ['sample', 'sample_type', 'project_id']].drop_duplicates(['sample'])
    phe1['sample'] =  phe1.apply(lambda x : x['sample'][:-1], axis=1)
    phe2 = pd.read_csv("https://tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz", sep="\t")
    ph_join = pd.merge(left = phe2 , right = phe1, how = "left", on = "sample").dropna(subset=['project_id'])
    
    if cancer_type == "PAN" or cancer_type == "PANCAN":
        filterd = ph_join.loc[ph_join['sample_type_y'] == "Primary Tumor", :]
        sample_barcode = filterd["sample"].tolist()
    else:
        filterd = ph_join.loc[(ph_join['sample_type_y'] == "Primary Tumor") & (ph_join['project_id'] == "TCGA-" + cancer_type) , :]
        sample_barcode = filterd["sample"].tolist()
        
    intersect_ = list(set(cols).intersection(sample_barcode))
    
    return intersect_

# dataset 불러오기
def get_dataset(path, std):
    # non_scale dataset
    # load raw DF
    DF = pd.read_csv(path, sep = "\t")
    pr_sample_DF = DF.iloc[:, 0:1]

    DF = DF.drop(["Sample_barcode"], axis=1)
        
    # remove outlier
    DF = replace_with_thresholds_iqr(dataframe=DF, cols=DF.columns)
    max_number_of_nas = DF.shape[0] / 2
    DF = DF.loc[:, (DF.isnull().sum(axis=0) <= max_number_of_nas)]

    # normalization
    if std:
        scalerX = StandardScaler()
        scalerX.fit(DF)
        DF_sacle = scalerX.transform(DF)
    else :
        scalerX = MinMaxScaler()
        scalerX.fit(X_train)
        DF_sacle = scalerX.transform(DF)
    
    # missing impute
    imputer = KNNImputer(n_neighbors=10)
    DF_sacle_impute = imputer.fit_transform(DF_sacle)

    DF_sacle_impute = pd.DataFrame(DF_sacle_impute, columns=DF.columns)

    return DF_sacle_impute


# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# 1. ML을 위한 metric 계산
# Autokeras
# import kerastuner
# import autokeras as ak
# from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# data.frame을 7:3 비율로 train set과 test set으로 구분
def get_train_test_DF(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, shuffle=True, random_state=32, stratify=y)
    
    return X_train, X_test, y_train, y_test


# 2. AutoML 돌리기
# mljar
# evaluate a model using repeated k-fold cross-validation
def evaluate_model_with_automl(X_train, X_test, Y_train, Y_test, path, m):    
    # run automl
    automl = AutoML(results_path=path, mode=m, ml_task="binary_classification",total_time_limit=30000)
    automl.fit(X_train, Y_train)
       
    # Test Accuracy
    predictions = automl.predict_all(X_test)
    cm = confusion_matrix(Y_test, predictions["label"].astype(int))
    plot_confusion_matrix(cm=cm, y_true=Y_test, y_pred=predictions["label"].astype(int), normalize=False, labels=True, target_names=["Non-Response", "Response"], 
             path=path)
    
# 3. Confusion matrix (negative / positive plot)
def plot_confusion_matrix(path, cm, y_true, y_pred, target_names=None, cmap=None, normalize=True, labels=True, title='Confusion matrix'):
    
    f1 = f1_score(y_true=y_true, y_pred=y_pred)
    mcc = matthews_corrcoef(y_true=y_true, y_pred=y_pred)
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    
    #accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)
    
    if labels:
        for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
            if normalize:
                plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                         horizontalalignment="center",
                         color="white" if cm[i, j] > thresh else "black")
            else:
                plt.text(j, i, "{:,}".format(cm[i, j]),
                         horizontalalignment="center",
                         color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\nf1-score={:0.4f};mcc={:0.4f};accuracy={:0.4f}; misclass={:0.4f}'.format(f1, mcc, accuracy, misclass))
    plt.savefig(path + '/Test_CM_Plot.png', dpi=80, bbox_inches="tight")

# 폴더 만들기
def createFolder(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print ('Folder already exists. ' +  path)
        

def cancer_select(df, cancer_type):
    # phenotype
    phe1 = pd.read_csv("https://gdc-hub.s3.us-east-1.amazonaws.com/download/GDC-PANCAN.basic_phenotype.tsv.gz", sep="\t")
    phe1 = phe1.loc[phe1.program == "TCGA", :].loc[:, ['sample', 'sample_type', 'project_id']].drop_duplicates(['sample'])
    phe1['sample'] =  phe1.apply(lambda x : x['sample'][:-1], axis=1)
    phe2 = pd.read_csv("https://tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz", sep="\t")
    ph_join = pd.merge(left = phe2 , right = phe1, how = "left", on = "sample").dropna(subset=['project_id'])
    
    if cancer_type == "PAN" or cancer_type == "PANCAN":
        filterd = ph_join.loc[ph_join['sample_type_y'] == "Primary Tumor", :]
        sample_barcode = filterd["sample"].tolist()
    else:
        filterd = ph_join.loc[(ph_join['sample_type_y'] == "Primary Tumor") & (ph_join['project_id'] == "TCGA-" + cancer_type) , :]
        sample_barcode = filterd["sample"].tolist()
        
    intersect_ = list(set(df.iloc[:, 0].to_list()).intersection(sample_barcode))
    
    isin_filter = df.iloc[:, 0].isin(intersect_)
    
    DF_intersect = df[isin_filter]
    
    return DF_intersect

### **N2D approch for data preprocessing**|

In [3]:
DF = get_dataset(path="D:/Desktop/Machine_Learning/rawdata/autoencoder/LUSC/LUSC.txt",std=True)
DF_barcode = pd.read_csv("D:/Desktop/Machine_Learning/rawdata/autoencoder/LUSC/LUSC.txt", sep = "\t",usecols = ["Sample_barcode"] )


In [4]:
# train set /test set 생성
X_train, X_test = train_test_split(DF, test_size = .3, random_state = 21, shuffle=True)

In [5]:
DF_total = pd.concat([DF_barcode,DF],axis =1)

In [6]:
X_train.to_csv("D:/Desktop/Machine_Learning/rawdata/autoencoder/LUSC/X_train.txt",sep = '\t', mode='w')
X_test.to_csv("D:/Desktop/Machine_Learning/rawdata/autoencoder/LUSC/X_test.txt",sep = '\t', mode='w')

## **RUN Autoencoder**

### **Bottleneck layer로 Autoencoder 진행**

###  **Earlystopping 과정으로 epoch 줄이기**

### **Validataion loss plot 그려서 확인**

# **Clustering 진행**

### Encoding 된 data로 tsne 실행

### Autoencoder+TSNE -> KM clustering 방법으로 visualization 진행

### KM_tsne grouping data 확인 및 저장

In [7]:
SAVE_PATH = "D:/Desktop/Machine_Learning/Pipeline_1/LUSC/AutoEncoder"

In [8]:
for index in range(5000):
    
    try:
        file_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
    
    
      
    
    # ## **RUN Autoencoder**

    # ### **Bottleneck layer로 Autoencoder 진행**

    

        inputs_dim = X_train.shape[1]
        encoder = Input(shape = (inputs_dim, ))
        e = Dense(512, activation = "relu")(encoder)
        e = Dense(256, activation = "relu")(e)
        e = Dense(128, activation = "relu")(e)
        ## bottleneck layer
        n_bottleneck = 50
        ## defining it with a name to extract it later
        bottleneck_layer = "bottleneck_layer"
        # can also be defined with an activation function, relu for instance
        bottleneck = Dense(n_bottleneck, name = bottleneck_layer)(e)
        ## define the decoder (in reverse)
        decoder = Dense(128, activation = "relu")(bottleneck)
        decoder = Dense(256, activation = "relu")(decoder)
        decoder = Dense(512, activation = "relu")(decoder)
        ## output layer
        output = Dense(inputs_dim)(decoder)
        ## model
        model = Model(inputs = encoder, outputs = output)

        encoder = Model(inputs = model.input, outputs = bottleneck)


        # ###  **Earlystopping 과정으로 epoch 줄이기**



        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
        model.compile(loss = "mse", optimizer = "adam")
        history = model.fit(
            X_train,
            X_train,
            batch_size = 32,
            epochs = 30,
            verbose = 0,
            validation_data = (X_test, X_test),
            callbacks=[es]
        )

        # verbose = 1 : earlystopping 과정 보여주기 / 0 : 보여주지 않기


        









        DF_encoded = encoder.predict(DF)




        # # **Clustering 진행**

        # ### Encoding 된 data로 tsne 실행




        tsne = TSNE(2, metric = "euclidean", random_state = 13).fit_transform(DF_encoded)


        # ### Autoencoder+TSNE -> KM clustering 방법으로 visualization 진행




        kmeans = KMeans(n_clusters = 2, random_state = 331, max_iter = 1000)
        km_tsne = kmeans.fit_predict(tsne)

        kmtsne = pd.DataFrame(km_tsne, columns = ['km_tsne'])

        ###### validation (Silhouette / SVM)






        ######### SVM

        df_valid = pd.concat([DF, kmtsne], axis=1)

        A_train, A_test , B_train, B_test = train_test_split(df_valid, km_tsne, test_size = .3, random_state = 21, shuffle=True)
        C_train, C_test , D_train, D_test = train_test_split(df_valid, km_tsne, test_size = .3, random_state = 21, shuffle=True)

        # ANOVA SVM
        # 1) anova 필터, 300 개의 최고 순위 기능 가져 오기
        anova_filter = SelectKBest(f_classif, k=200)
        # 2) svm
        clf = svm.LinearSVC()

        anova_svm = make_pipeline(anova_filter, clf)
        anova_svm.fit(A_train, B_train)

        B_pred = anova_svm.predict(A_test)
        accur_anova = metrics.accuracy_score(B_test, B_pred)

        list_data_anova =  ["km_tsne_"+ str(index+1), str(accur_anova)]
        if index == 0:
            with open(SAVE_PATH + "/group_anova.csv", "w", newline='') as f:
                writer = csv.writer(f)
                writer.writerow(list_data_anova)
        else :
            with open(SAVE_PATH + "/group_anova.csv", "a", newline='') as f:
                writer = csv.writer(f)
                writer.writerow(list_data_anova)

        # Randomforest SVM
        rf_filter = SelectFromModel(RandomForestClassifier(n_estimators = 200))
        rf_svm = make_pipeline(rf_filter, clf)
        rf_svm.fit(C_train, D_train)

        D_pred = rf_svm.predict(C_test)
        accur_rf = metrics.accuracy_score(D_test,D_pred)

        list_data_rf =  ["km_tsne_"+str(index+1), str(accur_rf)]        
        if index == 0:
            with open(SAVE_PATH + "/group_rf.csv", "w", newline='') as f:
                writer = csv.writer(f)
                writer.writerow(list_data_rf)
        else :
            with open(SAVE_PATH + "/group_rf.csv", "a", newline='') as f:
                writer = csv.writer(f)
                writer.writerow(list_data_rf)        

          #########Silhouette

        silhouette_avg = silhouette_score(tsne, km_tsne)
        print(
            "The average silhouette_score is :",
            silhouette_avg,
        )

        list_data_sil =  ["km_tsne_"+ str(index+1), str(silhouette_avg)]

        if index == 0:
            with open(SAVE_PATH + "/group_silhouette.csv", "w", newline='') as f:
                writer = csv.writer(f)
                writer.writerow(list_data_sil)
        else :
            with open(SAVE_PATH + "/group_silhouette.csv", "a", newline='') as f:
                writer = csv.writer(f)
                writer.writerow(list_data_sil)


        ########### scatter_plot        
        LABEL_COLOR_MAP = {0 : 'r',
                       1 : 'b'
                          }
        label_color = [LABEL_COLOR_MAP[l] for l in km_tsne]
        plt.scatter(tsne[:, 0], tsne[:, 1], c = label_color, s = 1)
        plt.savefig("D:/Desktop/Machine_Learning/Pipeline_1/LUSC/AutoEncoder/clusterplot/" + file_name + "_Pipeline1_clusterplot.png")
        plt.close()

        # ### **Validataion loss plot 그려서 확인**



        history_dict = history.history
        loss_values = history_dict["loss"]
        val_loss_values = history_dict["val_loss"]
        epochs = history.epoch
        plt.plot(epochs, loss_values, "o", label = "training loss")
        plt.plot(epochs, val_loss_values, "b", label = "validation loss")
        plt.title("Training & validation loss")
        plt.xlabel("epoch")
        plt.ylabel("loss")
        plt.legend()
        plt.savefig("D:/Desktop/Machine_Learning/Pipeline_1/LUSC/AutoEncoder/vlp/" + file_name + "_Pipeline1_vlp.png")
        plt.close()

        # ### KM_tsne grouping data 확인 및 저장


        model.save("D:/Desktop/Machine_Learning/Pipeline_1/LUSC/AutoEncoder/model/" + file_name + "_Pipeline1_AE.h5")

        dummytsne = pd.concat([DF_barcode,kmtsne],axis=1)


        dummytsne.to_csv("D:/Desktop/Machine_Learning/Pipeline_1/LUSC/AutoEncoder/group/" + file_name + "_Pipeline1_group_result.txt",sep = '\t', mode='w')
        
    except ValueError:
        pass

Epoch 00014: early stopping


The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
divide by zero encountered in true_divide


The average silhouette_score is : 0.4953182
Epoch 00017: early stopping


The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
divide by zero encountered in true_divide


The average silhouette_score is : 0.5117686
Epoch 00017: early stopping


...

The average silhouette_score is : 0.4856899
Epoch 00018: early stopping


The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
divide by zero encountered in true_divide


The average silhouette_score is : 0.48323536
Epoch 00017: early stopping


The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
divide by zero encountered in true_divide


The average silhouette_score is : 0.47359625
